In [1]:
# Importing Libraries
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/manan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/manan/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


In [2]:
#pip install -U langchain-community


In [3]:
#!python3 -m ensurepip --upgrade
#!python3 -m pip install --upgrade pip
#!python3 -m pip install langchain langchain-core langchain-google-genai

In [4]:
# Connect MySQL database
host = '127.0.0.1'
port = '3306'
username = 'root'
password = 'Pikachu@12345'
database_schema = 'text_to_sql'

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [5]:
from urllib.parse import quote_plus

password = quote_plus("Pikachu@12345")  # encodes @ as %40
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [6]:
#pip install cryptography

In [7]:
db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=1)
db.get_table_info()

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n1 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n1 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB\n\n/*\n1 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thouseholds INTEGER, \n\tmedian_income INTEGER, \n\tland_area INTEGER, \

In [8]:
#pip install pymysql

In [9]:
# Create the LLM Prompt Template
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query dont include anything else. Provide me sql query in a single line dont add line breaks
Table Schema: {schema}
Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = 'YOUR_GOOGLE_GENAI_API_KEY')

In [17]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema=lambda _:get_schema(db)) 
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [18]:
# test the SQL query chain with a sample question

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)

```sql
SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = 'Geiss Company'
```
